In [ ]:
import pandas as pd
import os
from utils import get_mean_results
import numpy as np
import matplotlib.pyplot as plt
from dcurves import dca as decision_curve_analysis
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['savefig.dpi'] = 500
# set xticks fontsize
plt.rcParams['xtick.labelsize'] = 18
# set yticks fontsize
plt.rcParams['ytick.labelsize'] = 18
# set labels fontsize
plt.rcParams['axes.labelsize'] = 22
# set legend fontsize
plt.rcParams['legend.fontsize'] = 12
# set labels weight to bold
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['figure.figsize'] = (10, 10)

In [ ]:
thresholds = np.linspace(0, 1, 101)

In [ ]:
path = {
    '../tests/test1/train-PT-test-US' : 'All Features',
    '../tests/test1/train-PT-test-US-reduced-13' : '13 Best Features',
}
save_name = 'train-PT-test-US-models'

In [ ]:
def get_dca_values(path):
    dca_values = {
        'y_proba_1': [],
        'all' : [],
        'none' : [],
        'thresholds' : []
    }
    for exp in os.listdir(path):
        preds_path = os.path.join(path, exp, 'predictions')
        for preds in [preds for preds in os.listdir(preds_path) if 'LogisticRegression' in preds]:
            predictions = pd.read_csv(os.path.join(preds_path, preds))
            dca_df = decision_curve_analysis(
                data = predictions,
                outcome = 'y_true',
                modelnames = ['y_proba_1'],
                thresholds = thresholds
            )
            # save values
            for model in dca_df.model.unique():
                dca_values[model].append(dca_df.loc[dca_df['model'] == model, 'net_benefit'].values)
                dca_values['thresholds'].append(dca_df.loc[dca_df['model'] == model, 'threshold'].values)

    for key, value in dca_values.items():
        print(key)
        dca_values[key] = np.mean(value, axis=0)
    return dca_values

In [ ]:
values = {}
for path_dir, title in path.items():
    dca_values = get_dca_values(path_dir)
    values[title] = dca_values

In [ ]:
names = {
    'y_proba_1' : 'LR',
    'all' : 'Treat as C-Section',
    'none' : 'Treat as Vaginal Delivery'
}

In [ ]:
plt.figure(figsize=(10, 6))
dca_values = list(values.values())[0]
cmap = plt.get_cmap('Greys')
plt.plot(dca_values['thresholds'], dca_values['all'], label=f'{names["all"]}', linewidth=2, color=cmap(0.8))
plt.plot(dca_values['thresholds'], dca_values['none'], label=f'{names["none"]}', linewidth=2, color=cmap(0.6))
# setup colors
for title, dca_values in values.items():
    plt.plot(dca_values['thresholds'], dca_values['y_proba_1'], label=f'{names["y_proba_1"]} - {title}', linewidth=3)
plt.xlabel('Threshold')
plt.ylabel('Net Benefit')
plt.xticks(np.linspace(0, 1, 11))
plt.yticks(np.arange(-0.05, 0.35, 0.05))
plt.ylim([-0.05, 0.35])
plt.xlim([0, 1])
plt.grid(linestyle='--', linewidth=1, alpha=0.7)
plt.legend()
plt.savefig(f'../images/dca_{save_name}.png', bbox_inches='tight')
plt.savefig(f'../images/dca_{save_name}.svg', bbox_inches='tight')
plt.savefig(f'../images/dca_{save_name}.pdf', bbox_inches='tight')
plt.show()